In [2]:
#%pip install python-docx
#%pip install git+https://github.com/python-openxml/cxml.git
#%pip install docx2txt
#%pip install aspose-words
#%pip install docx2pdf
%pip install ipywidgets


  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/70/1a/7edeedb1c089d63ccd8bd5c0612334774e90cf9337de9fe6c82d90081791/ipywidgets-8.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.10 from https://files.pythonhosted.org/packages/99/bc/82a8c3985209ca7c0a61b383c80e015fd92e74f8ba0ec1af98f9d6ca8dce/widgetsnbextension-4.0.10-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.10 from https://files.pythonhosted.org/packages/24/da/db1cb0387a7e4086780aff137987ee924e953d7f91b2a870f994b9b1eeb8/jupyterlab_widgets-3.0.10-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/139.4 kB ? eta -:--:--
   ----------------- --------------------- 61.4/139.4 kB 656.4 kB/s eta 0:00:01
   ---------------------------------------- 139.4/139.4 kB 1.2 MB/s eta 0:00:00
   -----------------------------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import docx
from docx import Document
from cxml import xml
import os
import docx2txt                           # Extract images
from docx.text.paragraph import Paragraph
import zipfile
import io
import base64
from PIL import Image
from docx.shared import Inches
import aspose.words as aw
import docx2pdf

In [4]:
os.getcwd()

'c:\\Users\\A485205\\Desktop\\Hackathon\\openai-hackathon'

In [2]:
os.chdir('C:\\Users\\A485205\\Desktop\\Hackathon\\openai-hackathon')

# Docx path


In [10]:
# Usage
# doc_path = './input_docx/P6700 ITM Vol 1 - Intro, Scope, Competition.docx'
# folder_name = './output_docx/P6700 ITM Vol 1 - Intro, Scope, Competition'

#doc_path = './input_docx/P6700 ITM Vol 2 - Design, Market, and Values.docx'

#doc_path = './input_docx/P6700 ITM Vol 3 - Feature Targets, Testing.docx'

#doc_path = './input_docx/P6700 ITM Vol 4 - Platform, Models, Packages, Adaptation.docx' 

#doc_path = './input_docx/P6700 ITM Vol 5 - Advanced Driver Assistance Safety Systems.docx' 

doc_path = './input_docx/P6700 ITM Vol 6 - Powertrain, Programmable Features.docx'     # Error?
folder_name = './output_docx/P6700 ITM Vol 6 - Powertrain, Programmable Features' 

# doc_path = './input_docx/P6700 ITM Vol 7 - Axles, Chassis, Trailer Body Connections, Tires & Wheels.docx' 
# folder_name = './output_docx/P6700 ITM Vol 7 - Axles, Chassis, Trailer Body Connections, Tires & Wheels'

# doc_path = './input_docx/P6700 ITM Vol 8 - Cab, Lighting, Audio, Equipment.docx'
# folder_name = './output_docx/P6700 ITM Vol 8 - Cab, Lighting, Audio, Equipment'

# doc_path = './input_docx/P6700 ITM Vol 9 - Business Services, Aftermarket, Warranty.docx' 
# folder_name = './output_docx/P6700 ITM Vol 9 - Business Services, Aftermarket, Warranty' 

# os.path.exists(doc_path)
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    


#folder_name = './output_docx/P6700 ITM Vol 2 - Design, Market, and Values'
#folder_name = './output_docx/P6700 ITM Vol 3 - Feature Targets, Testing'
#folder_name = './output_docx/P6700 ITM Vol 4 - Platform, Models, Packages, Adaptation' 
#folder_name = './output_docx/P6700 ITM Vol 5 - Advanced Driver Assistance Safety Systems'

# Pdf paths

In [ ]:
doc_path = './input_docx/P6700 ITM Vol 6 - Powertrain, Programmable Features.pdf'     # Error?
folder_name = './output_docx/P6700 ITM Vol 6 - Powertrain, Programmable Features' 

# Spire

In [10]:
import tempfile
from typing import Generator, Tuple, Union

from docx import Document
from docx.document import Document as DocType
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P
from docx.oxml.xmlchemy import BaseOxmlElement
from docx.text.paragraph import Paragraph

In [11]:
def iterparts(doc_path:str, skip_first=True, bias:int=0) -> Generator[Tuple[int,DocType],None,None]:
    """Iterate over sub-documents by splitting source document into parts
    Split into parts by copying source document and cutting off unrelevant
    data.

    Args:
        doc_path (str):                 path to source *docx* file
        skip_first (bool, optional):    skip first split point and wait for 
                                        second occurrence. Defaults to True.
        bias (int, optional):           split point bias. Defaults to 0.

    Yields:
        Generator[Tuple[int,DocType],None,None]:    first element of each tuple 
                                                    indicates the number of a 
                                                    sub-document, if number is 0 
                                                    then there are no sub-documents
    """
    doc = Document(doc_path)
    counter = 0
    while doc:
        split_elem_idx = -1
        doc_body = doc.element.body
        cutted = [doc, None]
        for idx, elem in enumerate(doc_body.iterchildren()):
            if is_split_point(elem):
                if split_elem_idx == -1 and skip_first:
                    split_elem_idx = idx
                else:
                    cutted = split(doc, idx+bias) # idx-1 to keep previous paragraph
                    counter += 1
                    break
        yield (counter, cutted[0])
        doc = cutted[1]

def is_split_point(element:BaseOxmlElement) -> bool:
    """Split criteria

    Args:
        element (BaseOxmlElement): oxml element

    Returns:
        bool: whether the *element* is the beginning of a new sub-document
    """
    if isinstance(element, CT_P):
        p = Paragraph(element, element.getparent())
        return p.text.startswith("Some text")
    return False

def split(doc:DocType, cut_idx:int) -> Tuple[DocType,DocType]:
    """Splitting into parts by copying source document and cutting of
    unrelevant data.

    Args:
        doc (DocType): [description]
        cut_idx (int): [description]

    Returns:
        Tuple[DocType,DocType]: [description]
    """
    tmpdocfile = write_tmp_doc(doc)
    second_part = doc
    second_elems = list(second_part.element.body.iterchildren())
    for i in range(0, cut_idx):
        remove_element(second_elems[i])
    first_part = Document(tmpdocfile)
    first_elems = list(first_part.element.body.iterchildren())
    for i in range(cut_idx, len(first_elems)):
        remove_element(first_elems[i])
    tmpdocfile.close()
    return (first_part, second_part)

def remove_element(elem: Union[CT_P,CT_Tbl]):
    elem.getparent().remove(elem)

def write_tmp_doc(doc:DocType):
    tmp = tempfile.TemporaryFile()
    doc.save(tmp)
    return tmp

# Convert docx to pdf and split

In [111]:
from docx2pdf import convert

In [115]:
convert(doc_path, doc_path.replace('docx','pdf'))

  0%|          | 0/1 [00:00<?, ?it/s]

# Filling a docx template with Python while preserving style
https://blog.xa0.de/post/Filling-a-docx-template-with-Python-while-preserving-style/

In [ ]:
from docx import Document
# Load your document
doc = Document(doc_path)

In [ ]:
# Mock data for placeholders
mock_data = {
    "PLACEHOLDER1": "replacement text",
    "PLACEHOLDER2": "another piece of text"
    # Add as many placeholders as needed
}

In [11]:
# Function to replace text without changing style
def replace_text_preserve_style(paragraph, key, value):
    if key in paragraph.text:
        inline = paragraph.runs
        for i in range(len(inline)):
            if key in inline[i].text:
                text = inline[i].text.replace(key, value)
                inline[i].text = text

# Replace placeholders with mock data
for paragraph in doc.paragraphs:
    for key in mock_data:
        replace_text_preserve_style(paragraph, key, mock_data[key])

In [ ]:
# Save the modified document
doc.save(f'{folder_name}/path_to_modified_document.docx')

# Using aspose.words

In [9]:
doc = aw.Document(doc_path)
pageCount = doc.page_count
pageCount

8

In [10]:
# Save the document to HTML format.
doc.save(f"{folder_name}/test.pdf")

In [104]:
# extract range of pages
extractedPages = doc.extract_pages(7, 1)

# save pages as a separate document
extractedPages.save(f"{folder_name}/split_by_page_range.docx")

In [88]:
# load Word document
doc = aw.Document(doc_path)

for i in range(0, doc.sections.count) :
            
    # clone the section to split
    section = doc.sections[i].clone()

    # create an instance of Document class for new doucment
    newDoc = aw.Document()
    
    # clear the default sections
    newDoc.sections.clear()

    # inster section into new document
    newSection = newDoc.import_node(section, True).as_section()
    newDoc.sections.add(newSection)


    # Save section as a separate document
    newDoc.save(f"folder_name/Sec{i}.docx")

# Docx as a zipfile

In [8]:
with zipfile.ZipFile(doc_path, 'r') as zip_ref:
    zip_ref.extractall('./output_docx/test')


In [ ]:
archive = zipfile.ZipFile(doc_path)
for file in archive.filelist:
    archive.extract(file, 'extracted_docx')
#for file in os.listdir('./output_docx/text'):
    if file.endswith('.emf'):
        


    

# Extract number, name headings and their page numbers

In [3]:
# Extract number and name headings       
def extract_number_heading(doc_path: str):
    '''
    Input: path of a docx file 
    Output: a list of heading names and their number headings corresponding chapters, sections, subsections,... 
    '''
    doc = Document(doc_path)
    # Extract chapters, section, subsection numbers
    heading_numbers=[]
    heading_name=[]

    for paragraph in doc.paragraphs:
        # Check if the paragraph is a heading
        if paragraph.style.name.startswith('Heading'):            # Heading for all chapters, sections, subsections,.., Heading 1 for chapter, Heading 2 for subsections, Heading 3 for subsubsections
            # Save heading_numbers       
            heading_numbers.append(paragraph.style.name.split()[1])
            heading_name.append((paragraph.text))
    #print(heading_numbers)
    
    # Map heading_numbers to heading sections
    heading_sections_result = []
    chapter = 1
    for i in range(len(heading_numbers)):
        if heading_numbers[i] =='1':
            heading_sections_result.append(str(chapter))
            chapter+=1
        else:
            if int(heading_numbers[i]) > int(heading_numbers[i-1]):

                if int(heading_numbers[i]) == 2:
                    heading_sections_result.append(heading_sections_result[-1]+'.1')
                else:
                    heading_sections_result.append(heading_sections_result[-1] + '.1'*(int(heading_numbers[i])-2))


            elif int(heading_numbers[i]) == int(heading_numbers[i-1]):
                u = heading_sections_result[-1].split('.')
                u[-1] = str(int(u[-1])+1)
                heading_sections_result.append('.'.join(u))
       
            elif int(heading_numbers[i]) < int(heading_numbers[i-1]):
                u = heading_sections_result[-1].split('.')
                u = u[:int(heading_numbers[i])]
                u[-1] = str(int(u[-1])+1)
                heading_sections_result.append('.'.join(u))
    
    # Combine heading_name and heading_sections_result 
    total_heading = []
    for i in range(len(heading_numbers)):
        total_heading.append(heading_sections_result[i] + ' ' + heading_name[i])

    return total_heading

# Splitting documents and keeping images

In [81]:
# Splitting documents and keeping images
def save_content_under_headings(doc_path, folder_name):

    
    # Load the Word document
    doc = Document(doc_path)

    errors = []

    
    # Create a new folder
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    current_heading = None
    current_doc = None
    
    doc_name = doc_path.split('/')[-1]
    doc_name = '['+ ''.join(doc_name.split(".")[0]) +']' + ' '
    heading_name_list = extract_number_heading(doc_path)
    k=-1

    # Create a new folder images
    path_folder_images = folder_name + '_images'
    if not os.path.exists(path_folder_images):
        os.makedirs(path_folder_images)

    # Extract images
    # with zipfile.ZipFile(doc_path, 'r') as zip_ref:
    #     zip_ref.extractall(path_folder_images)
    #docx2txt.process(doc_path, path_folder_images)

    # Images name
    rels = {}
    for r in doc.part.rels.values():
        if isinstance(r._target, docx.parts.image.ImagePart):
            if ('.png' in r._target.partname) or ('.jpg' in r._target.partname) or ('.emf' in r._target.partname):
                rels[r.rId] = os.path.basename(r._target.partname)
    rels = dict(sorted(rels.items()))
    #print(rels)

    # Start splitting
    for paragraph in doc.paragraphs:
        # Check if the paragraph is a heading
        if paragraph.style.name.startswith('Heading'):            # Heading for all chapters, sections, subsections,.., Heading 1 for chapter, Heading 2 for subsections, Heading 3 for subsubsections
            # Save the previous section if there is one           
            #print(paragraph.style.name.split()[1])

            # Advoid special characters in headings to save file
            current_heading = heading_name_list[k]
            current_heading = current_heading.replace('/', '-').replace('\\', '-').replace('\t','')

            for s in ['*', '"', '<', '>', '|', '?', ':']:
                current_heading = current_heading.replace(s, '')

            k+=1

            if current_heading and current_doc:
                file_path = os.path.join(folder_name, f"{doc_name}{current_heading}.docx")

                try:
                    current_doc.save(file_path)
                except:
                    print('Errors file path:', file_path)
                    errors.append(current_heading)
                    pass
            
            # Reset for the next section
            current_heading = heading_name_list[k]
            current_heading = current_heading.replace('/', '-').replace('\\', '-').replace('\t','')
            current_doc = Document()

            current_doc.add_heading(current_heading, level=0)           

        else:
            # Accumulate content under the current heading
            if current_doc is not None:
                current_doc.add_paragraph(paragraph.text)

                # Input images
                #if 'graphicData' in paragraph._p.xml:
                if 'Graphic' in paragraph._p.xml:

                  
                    # Get the rId of the image
                    for rId in rels:
                        if rId in paragraph._p.xml:

                            path_image = path_folder_images + '\\word\\media' +'\\' + rels[rId]
                            
                            # with open(path_image, "rb") as f:
                            #     image = base64.b64encode(f.read()).decode("utf-8")                     
                            #     current_doc.add_picture(io.BytesIO(image))

                            if '.emf' in rels[rId]:
                                # # get sizes of the emf image
                                # with open(path_image, "rb") as f:
                                #     f.read(16)
                                #     w1, w2 = f.read(1).hex(), f.read(1).hex()
                                #     f.read(2)
                                #     h1, h2 = f.read(1).hex(), f.read(1).hex()

                                # width  = int(str(w2) + str(w1), 16) * 762
                                # height = int(str(h2) + str(h1), 16) * 762

                                path_image_png = path_folder_images + '\\word\\media' +'\\' + rels[rId].split('.')[0] + '.png'
                                Image.open(path_image).save(path_image_png)

                                # img = Image.open(path_image)

                                # img = img.resize((width, height), Image.LANCZOS)

                                # img.save(path_image_png)

                                current_doc.add_picture(path_image_png, width=Inches(0.5), height=Inches(2))
                            else:
                                current_doc.add_picture(path_image)


                            
    # Don't forget to save the last section
    if current_heading and current_doc:
        current_heading = heading_name_list[k]
        current_heading = current_heading.replace('/', '-').replace('\\', '-').replace('\t','')
        for s in ['*', '"', '<', '>', '|', '?', ':']:
            current_heading = current_heading.replace(s, '')

        file_path = os.path.join(folder_name, f"{current_heading}.docx")
        try:
            current_doc.save(file_path)
        except:
            errors.append(current_heading)
            print('Errors file path:', file_path)
            pass
    
    if len(errors)>1:
        file = open(os.path.join(folder_name, 'sections_errors.txt'), 'w')
        file.write('\n\n'.join(errors))
        file.close()
    
    # End of the function

In [82]:
save_content_under_headings(doc_path, folder_name)

Errors file path: ./output_docx/P6700 ITM Vol 6 - Powertrain, Programmable Features\[P6700 ITM Vol 6 - Powertrain, Programmable Features] 3.2 Transmission Electronics Package (295) Still ok except the shifter has changed. - JBB.docx
Errors file path: ./output_docx/P6700 ITM Vol 6 - Powertrain, Programmable Features\[P6700 ITM Vol 6 - Powertrain, Programmable Features] 3.9 Roll Direction Change Inhibit (E3E) (VHD ONLY) – To be inhibited on all B1 B2, to be introduced with Vocational in 2027.docx
Errors file path: ./output_docx/P6700 ITM Vol 6 - Powertrain, Programmable Features\[P6700 ITM Vol 6 - Powertrain, Programmable Features] 4.83 Additional Road Speed Limit (8KB) – will need to review for GHG VNL or VHD intro.docx


# Doing all docx

In [69]:
check = []
for file_name in os.listdir('./input_docx/'):
    doc_path = f'./input_docx/{file_name}'
    folder_name = f'./output_docx/{file_name.split(".")[0]}'

    check.append([doc_path, folder_name])

check = check[:-1]
check 


[['./input_docx/P6700 ITM Vol 6 - Powertrain, Programmable Features.docx',
  './output_docx/P6700 ITM Vol 6 - Powertrain, Programmable Features'],
 ['./input_docx/P6700 ITM Vol 7 - Axles, Chassis, Trailer Body Connections, Tires & Wheels.docx',
  './output_docx/P6700 ITM Vol 7 - Axles, Chassis, Trailer Body Connections, Tires & Wheels'],
 ['./input_docx/P6700 ITM Vol 8 - Cab, Lighting, Audio, Equipment.docx',
  './output_docx/P6700 ITM Vol 8 - Cab, Lighting, Audio, Equipment'],
 ['./input_docx/P6700 ITM Vol 9 - Business Services, Aftermarket, Warranty.docx',
  './output_docx/P6700 ITM Vol 9 - Business Services, Aftermarket, Warranty']]

In [63]:
# Extract images
for [doc_path, folder_name] in check:
    # Create a new folder images
    path_folder_images = folder_name + '_images'
    if not os.path.exists(path_folder_images):
        os.makedirs(path_folder_images)

    # Extract images
    with zipfile.ZipFile(doc_path, 'r') as zip_ref:
        zip_ref.extractall(path_folder_images)

In [68]:
for [doc_path, folder_name] in check:
    save_content_under_headings(doc_path, folder_name)
    

Successfully: [P6700 ITM Vol 1 - Intro, Scope, Competition] 5.5 Market Strategy.docx
Successfully: [P6700 ITM Vol 2 - Design, Market, and Values] 3.7 Focused Product Training Areas.docx
Successfully: [P6700 ITM Vol 3 - Feature Targets, Testing] 7.2.2 Sun Simulation Testing and Climate Testing .docx
Successfully: [P6700 ITM Vol 4 - Platform, Models, Packages, Adaptation] 5.9 GHG Application .docx
Successfully: [P6700 ITM Vol 5 - Advanced Driver Assistance Safety Systems] 1.18 Rear Monitoring, Class II & IV (W5E) V1.4 Intro.docx
Successfully: [P6700 ITM Vol 6 - Powertrain, Programmable Features] 4.95 Fleet Management System Gateway (MJX).docx
Successfully: [P6700 ITM Vol 7 - Axles, Chassis, Trailer Body Connections, Tires & Wheels] 8.3 Tire Pressure Monitoring (3CE).docx
Successfully: [P6700 ITM Vol 8 - Cab, Lighting, Audio, Equipment] 7.1 Cab Paint Scheme Guidelines.docx
Successfully: [P6700 ITM Vol 9 - Business Services, Aftermarket, Warranty] 7 Warranty.docx


In [ ]:
# Do: save_content_under_headings for all docx in a folder
for file_name in os.listdir('./input_docx/'):
    doc_path = f'./input_docx/{file_name}'
    folder_name = f'./output_docx/{file_name.split(".")[0]}'
    save_content_under_headings(doc_path, folder_name)

In [8]:
# if  not os.path.exists(folder_name):
#     os.makedirs(folder_name)
    
# text = docx2txt.process(doc_path, folder_name)

In [10]:
# Test
# doc = Document(doc_path)
# image_paragraphs = []   

# for paragraph in doc.paragraphs:
#     if 'graphicData' in paragraph._p.xml:
#     #if paragragph_contains_image(paragraph):
#         image_paragraphs.append(paragraph._p.xml)

# len(image_paragraphs)

24

In [11]:
# # Test
# doc = Document(doc_path)
# image_paragraphs = []   

# for paragraph in doc.paragraphs:
#     if 'Graphic' in paragraph._p.xml:
#     #if paragragph_contains_image(paragraph):
#         image_paragraphs.append(paragraph._p.xml)

# len(image_paragraphs)

24

In [ ]:
# doc = Document(doc_path)
# image_paragraphs = []
# for s in doc.inline_shapes:
#     image_paragraphs.append([s.height, s.width, s._inline.graphic.graphicData.pic.nvPicPr.cNvPr.name])

# len (image_paragraphs)

8

In [ ]:
# doc = Document(doc_path)

# paraGr = []             
# index = []            # index of images 

# par = doc.paragraphs
# for i in range(len(par)):
#      paraGr.append(par[i].text)
#      if 'graphicData' in par[i]._p.xml:
#          index.append(i)

# print(index)       

[0, 1, 5, 57, 60, 64, 69, 78, 87, 92, 98, 101, 104, 110, 116, 124, 160, 346, 385, 402, 418, 423, 434, 446]


In [ ]:
# Do: save_content_under_headings for all docx in a folder
# for file_name in os.listdir('./input_docx/'):
#     doc_path = f'./input_docx/{file_name}'
#     folder_name = f'./output_docx/{file_name.split(".")[0]}'
#     save_content_under_headings(doc_path, folder_name)

In [ ]:
extract_number_heading(doc_path)

['1 Executive Summary',
 '2 Document Guidelines for Contributors',
 '3 Introduction',
 '3.1 Historical and Project Background',
 '3.2 Industry Regulations, Vehicle Credit System  ',
 '3.3 Strategic Overview',
 '3.4 Project Overview',
 '3.4.1.1 Documentation & Specification Strategy for Producing GHG Credits',
 '3.4.1.2 Powertrain Installation',
 '3.4.1.3 Group Electrical Architecture',
 '3.4.2 Project profile',
 '4 Competitive Offer',
 '5 Project Scope (from Project Description document)',
 '5.1 Prerequisites',
 '5.2 Product offering (BPS)',
 '5.3 Forecasted volumes',
 '5.4 Introduction Timing (Note, this timing is fluid)',
 '5.5 Market Strategy']